In [22]:
!pip install nltk
!pip install datasketch

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.6 MB/s eta 0:00:001.3 MB/s eta 0:00:01

[notice] A new release of pip is available: 23.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/ajdin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ajdin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
import string
import email

punctuations = list(string.punctuation)
stopwords = set(nltk.corpus.stopwords.words('english'))
stemmer = nltk.PorterStemmer()

# Combine the different parts of the email into a flat list of strings
def flatten_to_string(parts):
    ret = []
    if type(parts) == str:
        ret.append(parts)
    elif type(parts) == list:
        for part in parts:
            ret += flatten_to_string(part)
    elif parts.get_content_type == 'text/plain':
        ret += parts.get_payload()
    return ret

In [11]:
# Extract subject and body text from a single email file
def extract_email_text(path):
    # Load a single email from an input file
    with open(path, errors='ignore') as f:
        msg = email.message_from_file(f)
    if not msg:
        return ""

    # Read the email subject
    subject = msg['Subject']
    if not subject:
        subject = ""

    # Read the email body
    body = ' '.join(m for m in flatten_to_string(msg.get_payload())
                    if type(m) == str)
    if not body:
        body = ""

    return subject + ' ' + body

# Process a single email file into stemmed tokens
def load(path):
    email_text = extract_email_text(path)
    if not email_text:
        return []

    # Tokenize the message
    tokens = nltk.word_tokenize(email_text)

    # Remove punctuation from tokens
    tokens = [i.strip("".join(punctuations)) for i in tokens
              if i not in punctuations]

    # Remove stopwords and stem tokens
    if len(tokens) > 2:
        return [stemmer.stem(w) for w in tokens if w not in stopwords]
    return []

In [12]:
import os

DATA_DIR = './data/trec07p/data/'
LABELS_FILE = './data/trec07p/full/index'
TRAINING_SET_RATIO = 0.7

labels = {}
spam_words = set()
ham_words = set()

# Read the labels
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()
        label, key = line.split()
        labels[key.split('/')[-1]] = 1 if label.lower() == 'ham' else 0

# Split corpus into training and test sets
filelist = os.listdir(DATA_DIR)
X_train = filelist[:int(len(filelist)*TRAINING_SET_RATIO)]
X_test = filelist[int(len(filelist)*TRAINING_SET_RATIO):]

for filename in X_train:
    path = os.path.join(DATA_DIR, filename)
    if filename in labels:
        label = labels[filename]
        stems = load(path)
        if not stems:
            continue
        if label == 1:
            ham_words.update(stems)
        elif label == 0:
            spam_words.update(stems)
        else:
            continue

blacklist = spam_words - ham_words


In [16]:
blacklist

{'ȫʒʹ',
 'species////c.///vespertinus//binomial///coccothraustes/vespertinus/binomial_authority////w.///coop',
 'with/the/exception/of/a/single/day/in/541',
 'wun',
 'cajamarca',
 '164861scdxfa',
 'visualrange.com/images/tuxes_4932/zihoy_2.jpg',
 'time=5/6/2007',
 'escalon',
 'h/pgsl/160/_submenuid/population_0/ds_name/null/_ci_nbr/null/qr_name/null/reg',
 'www.direct2drive.com/images/product/icon/3603.jpg',
 'unevencycle.com/t/kwo3xqld/6103=22',
 'gravir',
 '3dstyle47',
 'virtuou',
 'www.ascendclick.com/i/tehmxo/9d1fkahj/belin/fuvan1268.gif',
 '990689812497',
 'steeger',
 'www.4ulanesdealer.com/i/0lzste/hq1knhtn/sutut_8400/jilak_3.jpg',
 'tuckeri',
 'fr0m',
 'mana/jana',
 'ggamoh.floorman.hk',
 'fire-weaken',
 'www.geocities.com/dicksonwct84440',
 'kisss',
 'fݸΰ״ｏšô\U000f1c75硢̩/ŷӡϻ/졢žַز',
 'hoqu',
 '322nd',
 'billyboyflick-typ',
 'demetriu',
 'retention/rates/rose/dramat',
 'dcpt88a',
 'boycamper.com/t/qqgb3xqld/17094=22',
 'pjp',
 'nzer',
 'offers/promot',
 '22.8',
 'dewar',
 'c2_sta

In [23]:
from datasketch import MinHash, MinHashLSH

# Extract only spam files for inserting into the LSH matcher
spam_files = [x for x in X_train if labels[x] == 0]

# Initialize MinHashLSH matcher with a Jaccard
# threshold of 0.5 and 128 MinHash permutation functions
lsh = MinHashLSH(threshold=0.5, num_perm=128)

# Populate the LSH matcher with training spam MinHashes
for idx, f in enumerate(spam_files):
    minhash = MinHash(num_perm=128)
    stems = load(os.path.join(DATA_DIR, f))
    if len(stems) < 2: continue
    for s in stems:
        minhash.update(s.encode('utf-8'))
    lsh.insert(f, minhash)

#Now it’s time to have the LSH matcher predict labels for the test set:

def lsh_predict_label(stems):
    '''
    Queries the LSH matcher and returns:
        0 if predicted spam
        1 if predicted ham
       −1 if parsing error
    '''
    minhash = MinHash(num_perm=128)
    if len(stems) < 2:
        return -1
    
    for s in stems:
        minhash.update(s.encode('utf-8'))
    matches = lsh.query(minhash)
    if matches:
        return 0
    else:
        return 1

In [27]:
def read_email_files():
    X = []
    y = []
    for i in range(len(labels)):
        filename = 'inmail.' + str(i+1)
        email_str = extract_email_text(os.path.join(DATA_DIR, filename))
        X.append(email_str)
        y.append(labels[filename])
    return X, y

In [28]:
from sklearn.model_selection import train_test_split

X, y = read_email_files()

X_train, X_test, y_train, y_test, idx_train, idx_test = \
    train_test_split(X, y, range(len(y)),
    train_size=TRAINING_SET_RATIO, random_state=2)



In [29]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

In [30]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Initialize the classifier and make label predictions
mnb = MultinomialNB()
mnb.fit(X_train_vector, y_train)
y_pred = mnb.predict(X_test_vector)

# Print results
print('Accuracy {:.3f}'.format(accuracy_score(y_test, y_pred)))

Accuracy 0.956
